# It has to start with Little.

As this is our first time hands on the live data, we can either get from some sort of API or we can run the scraping script to do it for us. For now, we will scrap a little data from [this site](https://www.flightradar24.com).

Let's get started ∞

# 

In [29]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input

import plotly.graph_objs as go
import requests

In [6]:
app = dash.Dash()

app.layout = html.Div([
                html.Div([
                    html.Iframe(src="https://www.flightradar24.com/0,-171.61/2",
                                height=500, width=1200)
                ])
])

app.run_server()

The thing is, we wanna show the site, in our page. But here, the site has refused to connect when it is called inside Iframe tag. So we have to try another website.

After some searcing of the topic which can be best for this purpose, I fount this stock price [site](https://www.financialexpress.com/market/stock-market/reliance-industries-ltd-stock-price/). So let's do scrapping.

In [13]:
from bs4 import BeautifulSoup as bs
import requests

In [48]:
app = dash.Dash()
url = "https://in.investing.com/equities/reliance-industries-historical-data"
app.layout = html.Div([
                html.Div([
                    html.Iframe(src=url,
                                height=500, width=1200)
                ]),
                html.Div([
                    html.H1(id="price"),
                    dcc.Interval(interval=2000, id='caller')
                ])
])

@app.callback(Output("price", "children"),
             [Input("caller", "n_intervals")])
def update_price(_):
    response = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bs(response.content)
    price = soup.find("bdo", attrs={"class":"last-price-value"}).text
    return f"The current price is {price}"


app.run_server()

Great! We are getting the price! Let's plot that on the graph.

In [50]:
app = dash.Dash()
url = "https://in.investing.com/equities/reliance-industries-historical-data"

app.layout = html.Div([
                html.Div([
                    html.Iframe(src=url,
                                height=500, width=1200)
                ]),
                html.Div([
                    html.H1(id="price"),
                    dcc.Interval(interval=2000, id='caller')
                ]),
                html.Div([
                    dcc.Graph(id="show_it")
                ])
])

stock_prices = []
@app.callback(Output("price", "children"),
             [Input("caller", "n_intervals")])
def update_price(_):
    response = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bs(response.content)
    price = soup.find("bdo", attrs={"class":"last-price-value"}).text
    stock_prices.append(price)
    return f"The current price is {price}"


@app.callback(Output("show_it", "figure"),
             [Input("caller", "n_intervals")])
def update_figure(_):
    plot = [go.Scatter(x = list(range(len(stock_prices))),
                       y = stock_prices,
                       mode = "lines")]
    
    layout = go.Layout(title="Reliance Stock Data")
    figure = go.Figure(data=plot, layout=layout)
    return figure
    
app.run_server()

Works amazingly! I know, there is no any comments or syntax analysis there in the code, but you know, it is simple going. So I didn't. 

# 

# That's it!
Next up, We will learn to deploy our dashboard.